In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

2025-12-06 13:29:08.803403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765027748.996516      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765027749.050672      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
# ==============================================================================
# 1. KONFIGURASI
# ==============================================================================
FOLDER_DATA = "/kaggle/input/open-meteo-climate/open_meteo_climate/"
NAMA_FILE = "kebumen_75tahun_lengkap.csv"
PATH_FILE = os.path.join(FOLDER_DATA, NAMA_FILE)

In [3]:
# ==============================================================================
# 2. LOAD DATA & BERSIHKAN WAKTU
# ==============================================================================
def load_data(filepath):
    print(f"📂 Membaca file: {filepath}")
    if not os.path.exists(filepath):
        print("❌ File tidak ditemukan.")
        return None
    
    # Baca CSV
    df = pd.read_csv(filepath, index_col='date', parse_dates=True)
    
    # Perbaikan Zona Waktu (Agar tidak geser)
    # 1. Baca sebagai UTC
    df.index = pd.to_datetime(df.index, utc=True)
    # 2. Geser ke WIB
    df.index = df.index.tz_convert('Asia/Jakarta')
    # 3. Hapus label zona waktu (biar jadi jam dinding polos)
    df.index = df.index.tz_localize(None)
    
    df = df.sort_index()
    print(f"✅ Data Siap! Rentang: {df.index.min()} s.d {df.index.max()}")
    return df

df = load_data(PATH_FILE)

# Ambil data 10 tahun terakhir agar relevan dan training tidak terlalu lama
# (Silakan ubah tahunnya jika mau lebih banyak data)
df_ai = df.loc['1950':].copy()

📂 Membaca file: /kaggle/input/open-meteo-climate/open_meteo_climate/kebumen_75tahun_lengkap.csv
✅ Data Siap! Rentang: 1950-01-01 01:00:00 s.d 2025-12-05 23:00:00


In [4]:
df_ai.tail(10)

,temperature,humidity,dewpoint,rain_mm,wind_speed,wind_direction,pressure,weather_code
date,,,,,,,,
2025-12-05 14:00:00,28.638000,75.081850,23.788,0.1,10.261579,195.255170,1006.83260,51.0
2025-12-05 15:00:00,27.388000,78.835020,23.388,0.7,7.729527,207.758450,1006.92350,53.0
2025-12-05 16:00:00,26.138000,87.446830,23.888,0.8,3.396233,212.005340,1006.91440,53.0
2025-12-05 17:00:00,25.588000,87.927580,23.438,1.1,2.340000,337.380100,1007.40910,55.0
2025-12-05 18:00:00,24.737999,94.463745,23.788,1.6,2.526420,355.914460,1008.30096,61.0
2025-12-05 19:00:00,24.138000,95.587100,23.388,2.4,2.930188,312.510400,1009.39410,61.0
2025-12-05 20:00:00,23.888000,97.915070,23.538,2.3,5.762812,38.659830,1009.99090,61.0
2025-12-05 21:00:00,23.938000,97.621240,23.538,0.9,3.438895,6.008918,1010.09125,53.0
2025-12-05 22:00:00,23.788000,98.803024,23.588,0.2,4.327493,343.072400,1010.09010,51.0


In [5]:
# ==============================================================================
# 3. FEATURE ENGINEERING (KELAS & MULTI-TIME LAG)
# ==============================================================================

# --- A. PEMBAGIAN KELAS (4 LEVEL) ---
def categorize_rain(mm):
    """Mengelompokkan curah hujan per jam menjadi 4 level."""
    if mm < 0.1:   return 0  # Tidak Hujan
    elif mm < 5.0: return 1  # Ringan
    elif mm < 10.0: return 2 # Sedang
    else:          return 3  # Lebat

# Terapkan kategori ke kolom target baru
df_ai['target_class'] = df_ai['rain_mm'].apply(categorize_rain)

# --- B. TIME LAG (FITUR MASA LALU UNTUK BANYAK KOLOM) ---
# 1. Tentukan kolom mana saja yang mau dibuatkan "memori"-nya
# Ganti nama kolom di bawah ini sesuai dengan nama kolom di data kamu!
fitur_untuk_lag = ['rain_mm', 'temperature', 'humidity', 'pressure'] 

# 2. Tentukan mau mundur berapa jam (misal: 1, 2, dan 3 jam lalu)
lags = [1, 2, 3, 4, 6, 12, 24] 

# 3. Loop otomatis untuk membuat kolom baru
for col in fitur_untuk_lag:
    # Cek dulu apakah kolomnya ada di data biar tidak error
    if col in df_ai.columns:
        for lag in lags:
            # Nama kolom baru: misal 'T2m_lag_1', 'RH_lag_2'
            col_name = f"{col}_lag_{lag}"
            df_ai[col_name] = df_ai[col].shift(lag)
    else:
        print(f"⚠️ Peringatan: Kolom '{col}' tidak ditemukan di data, dilewati.")

# --- C. PEMBERSIHAN (CLEANING) ---
# Hapus baris NaN akibat pergeseran data (shift)
df_ai = df_ai.dropna().reset_index(drop=True)

# --- CEK HASIL ---
print(f"\n✅ Total Data setelah lag & cleaning: {len(df_ai)} baris")
print(f"📊 Jumlah Fitur (Kolom) Sekarang: {len(df_ai.columns)}")

# Tampilkan contoh kolom T2m (Suhu) dan lag-nya
if 'T2m' in df_ai.columns:
    cols_show = ['T2m', 'T2m_lag_1', 'T2m_lag_2'] 
    print("\nContoh Lag pada Suhu (T2m):")
    print(df_ai[cols_show].head(5))


✅ Total Data setelah lag & cleaning: 665568 baris
📊 Jumlah Fitur (Kolom) Sekarang: 37


In [6]:
# ==============================================================================
# 4. PRE-PROCESSING (SCALING & WINDOWING)
# ==============================================================================
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# --- A. MENENTUKAN FITUR (X) & TARGET (y) ---

# 1. Fitur Utama (Sesuai request kamu)
fitur_utama = ['rain_mm', 'temperature', 'humidity', 'pressure']

# 2. Ambil otomatis semua kolom yang mengandung nama fitur utama + kolom lag-nya
# Ini trik biar kita tidak usah ngetik manual 'rain_lag_1', 'rain_lag_2', dst.
features = [col for col in df_ai.columns if any(x in col for x in fitur_utama)]

# 3. Target Output (Kita pakai target_class yang sudah dibuat di step 3)
target = 'target_class' 

print(f"✅ Jumlah Fitur Input (X): {len(features)} kolom")
print(f"   (Termasuk fitur asli dan semua lag yang telah dibuat)")
print(f"✅ Target Label (y): {target}")

# --- B. SCALING (MENYAMAKAN SKALA 0-1) ---
scaler = MinMaxScaler()
# Kita scale hanya kolom fitur (X), target (y) tidak perlu di-scale karena klasifikasi
X_scaled = scaler.fit_transform(df_ai[features])
y_raw = df_ai[target].values

# --- C. WINDOWING (MEMOTONG DATA BERDASARKAN WAKTU) ---
def create_windows(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# PENTING:
# Karena kita SUDAH punya fitur Lag (masa lalu) di kolom, 
# kita bisa set TIME_STEPS = 1 (atau angka kecil).
# Tapi jika kamu mau model melihat "urutan" data lag tersebut, kita bisa pakai angka lebih besar.
# Mari kita coba pakai 1 dulu karena kolomnya sudah mengandung informasi masa lalu (24 jam).
TIME_STEPS = 1 

print(f"\n✂️ Memotong data (Windowing) dengan Time Steps: {TIME_STEPS}...")
X_window, y_window = create_windows(X_scaled, y_raw, TIME_STEPS)

# --- D. SPLITTING DATA (LATIH & UJI) ---
# Split Training (80%) dan Testing (20%) - Tanpa mengacak urutan waktu
split_idx = int(len(X_window) * 0.80)

X_train, y_train = X_window[:split_idx], y_window[:split_idx]
X_test, y_test = X_window[split_idx:], y_window[split_idx:]

print(f"\n📊 Bentuk Data Siap Training:")
print(f"Data Latih (X_train): {X_train.shape}")
print(f"Label Latih (y_train): {y_train.shape}")
print(f"Data Uji  (X_test) : {X_test.shape}")

✅ Jumlah Fitur Input (X): 32 kolom
   (Termasuk fitur asli dan semua lag yang telah dibuat)
✅ Target Label (y): target_class

✂️ Memotong data (Windowing) dengan Time Steps: 1...

📊 Bentuk Data Siap Training:
Data Latih (X_train): (532453, 1, 32)
Label Latih (y_train): (532453,)
Data Uji  (X_test) : (133114, 1, 32)


In [12]:
# ==============================================================================
# 5. OVERSAMPLING (SMOTE)
# ==============================================================================
from imblearn.over_sampling import SMOTE
import numpy as np

print("\n⚖️ Sedang menyeimbangkan data dengan SMOTE...")

# Pastikan X_train dan y_train sudah ada di memori
# (Kalau error 'name X_train is not defined', berarti kamu lupa run sel di atasnya)

# 1. Flatten
n_samples, time_steps, n_features = X_train.shape
X_train_flat = X_train.reshape(n_samples, time_steps * n_features)

# 2. Resample
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)

# 3. Reshape kembali
X_train_balanced = X_train_resampled.reshape(-1, time_steps, n_features)
y_train_balanced = y_train_resampled

# Update variabel
X_train = X_train_balanced
y_train = y_train_balanced

print(f"✅ Data Selesai Diseimbangkan!")
print(f"   Data Baru (Sesudah): {np.bincount(y_train)}")

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7bc6bb677880>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error



⚖️ Sedang menyeimbangkan data dengan SMOTE...
✅ Data Selesai Diseimbangkan!
   Data Baru (Sesudah): [302075 302075 302075 302075]


In [15]:
# ==============================================================================
# 6. MEMBANGUN & MELATIH MODEL (ARSITEKTUR BARU)
# ==============================================================================
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# --- A. DEFINISI ARSITEKTUR MODEL ---
# Kita buat model baru yang bersih
model = Sequential()

# 1. Input Layer: Menyesuaikan bentuk data (Time Steps, Jumlah Fitur)
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

# 2. LSTM Layer: "Memori" untuk menangkap pola waktu
# return_sequences=False artinya kita cuma ambil kesimpulan di akhir waktu (bukan di setiap jam)
model.add(LSTM(64, return_sequences=False))

# 3. Dropout: Mencegah Overfitting (Biar gak cuma menghafal)
model.add(Dropout(0.3)) 

# 4. Dense Layer: Otak pemroses
model.add(Dense(32, activation='relu'))

# 5. Output Layer: 4 Neuron (sesuai 4 Kelas: 0, 1, 2, 3)
# Activation 'softmax' mengubah angka menjadi probabilitas (persentase)
model.add(Dense(4, activation='softmax')) 

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Wajib untuk klasifikasi multi-kelas integer (0,1,2,3)
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,044 (105.64 KB)

 Trainable params: 27,044 (105.64 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ==============================================================================
# 6. TRAINING (MELATIH OTAK AI)
# ==============================================================================
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# B. Reduce LR: Kalau stuck, turunkan learning rate biar lebih teliti
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)

# C. Checkpoint: Simpan model terbaik selama proses
checkpoint = ModelCheckpoint('model_terbaik.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr, checkpoint],
    verbose=1
)
# Plot History Belajar
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'], label='Akurasi Latih')
plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
plt.title('Grafik Pembelajaran Model')
plt.legend()
plt.show()

Epoch 1/100
8495/8496 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6822 - loss: 0.7022
Epoch 1: val_accuracy improved from -inf to 0.75067, saving model to model_terbaik.keras
8496/8496 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.6822 - loss: 0.7022 - val_accuracy: 0.7507 - val_loss: 0.5380 - learning_rate: 0.0010
Epoch 2/100
8494/8496 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7910 - loss: 0.5034
Epoch 2: val_accuracy improved from 0.75067 to 0.75141, saving model to model_terbaik.keras
8496/8496 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.7910 - loss: 0.5034 - val_accuracy: 0.7514 - val_loss: 0.5456 - learning_rate: 0.0010
Epoch 3/100
5156/8496 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.8046 - loss: 0.4801

In [ ]:
# ==============================================================================
# 7. EVALUASI HASIL (VERSI 4 KELAS)
# ==============================================================================
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print("\n📊 EVALUASI MODEL:")

# 1. Prediksi Data Test
y_pred_prob = model.predict(X_test)
y_pred_class = np.argmax(y_pred_prob, axis=1)

# 2. Definisikan Nama Kelas (4 Level)
# Pastikan urutannya sesuai: 0, 1, 2, 3
CLASSES = {
    0: 'Tidak Hujan', 
    1: 'Ringan', 
    2: 'Sedang', 
    3: 'Lebat'
}
target_names = list(CLASSES.values()) # Mengambil daftar nama saja

# 3. Laporan Klasifikasi
# Kita pakai range(4) bukan range(5) karena kelasnya cuma ada 4
print(classification_report(y_test, y_pred_class, target_names=target_names))

# 4. Confusion Matrix
cm = confusion_matrix(y_test, y_pred_class)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Prediksi AI')
plt.ylabel('Kenyataan (Aktual)')
plt.title('Confusion Matrix (Evaluasi Ketepatan)')
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight') 
print("✅ Gambar berhasil disimpan sebagai 'confusion_matrix.png'")
plt.show()

In [ ]:
# ==============================================================================
# 8. DEMO PREDIKSI (KARTU CUACA)
# ==============================================================================
def tampilkan_kartu_prediksi(input_24jam):
    """Menampilkan prediksi dengan gaya persentase."""
    input_reshaped = input_24jam.reshape(1, TIME_STEPS, len(features))
    probs = model.predict(input_reshaped, verbose=0)[0]
    
    pred_idx = np.argmax(probs)
    confidence = probs[pred_idx] * 100
    
    print("\n" + "="*50)
    print(f"🔮 PREDIKSI CUACA (1 Jam Kedepan)")
    print("="*50)
    
    print(f"Hasil Utama    : {CLASSES[pred_idx]}")
    print(f"Tingkat Yakin  : {confidence:.1f}%")
    print("-" * 50)
    
    print("Probabilitas:")
    for idx, label in CLASSES.items():
        score = probs[idx] * 100
        bar = "█" * int(score / 4)
        marker = "👈" if idx == pred_idx else ""
        print(f"{label:<30} : {score:>5.1f}% | {bar} {marker}")
    print("="*50)

# Coba cari data yang aslinya Hujan Lebat (Kelas 3 atau 4) untuk dites
print("\n🧪 DEMO HASIL PADA DATA BADAI:")
badai_indices = np.where(y_test >= 3)[0] # Cari kelas 3 atau 4

if len(badai_indices) > 0:
    # Ambil satu contoh
    sample_idx = badai_indices[0]
    tampilkan_kartu_prediksi(X_test[sample_idx])
else:
    print("Tidak ada data badai di set pengujian untuk didemokan.")